# Модуль 4.5 — Полный RAG‑пайплайн

**Цель:** собрать рабочий RAG: загрузка → эмбеддинги → ретривер → ответ модели с контекстом.

**Что сделаем:**
- установим библиотеки
- подготовим документы
- построим векторное хранилище
- сделаем retrieval‑chain и получим ответ

In [ ]:
%pip -q install -U \
  langchain \
  langchain-community \
  langchain-openai \
  chromadb \
  pydantic==2.12.3 \
  requests==2.32.4

## Если Colab ругается на зависимости

Иногда в Colab установлены библиотеки с жёсткими версиями (например, `google-adk` и `opentelemetry`).
Если после установки появляются конфликты — зафиксируйте совместимые версии ниже.

In [ ]:
%pip -q install -U \
  opentelemetry-api==1.37.0 \
  opentelemetry-sdk==1.37.0 \
  opentelemetry-proto==1.37.0 \
  opentelemetry-exporter-otlp-proto-common==1.37.0 \
  opentelemetry-exporter-otlp-proto-grpc==1.37.0

# На случай, если в окружении уже стоит более новая версия
def _force_pins():
    import sys, subprocess
    pkgs = [
        "opentelemetry-api==1.37.0",
        "opentelemetry-sdk==1.37.0",
        "opentelemetry-proto==1.37.0",
        "opentelemetry-exporter-otlp-proto-common==1.37.0",
        "opentelemetry-exporter-otlp-proto-grpc==1.37.0",
    ]
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-U", "--force-reinstall"] + pkgs)

_force_pins()

## Подготовка папки `data`

Если папки нет, создадим её и положим пример файла, чтобы всё запускалось сразу.

In [ ]:
from pathlib import Path

data_dir = Path("./data")
data_dir.mkdir(parents=True, exist_ok=True)

sample_path = data_dir / "sample.txt"
if not sample_path.exists():
    sample_path.write_text(
        "LangChain — это фреймворк для сборки LLM‑приложений.\n"
        "RAG (Retrieval‑Augmented Generation) улучшает ответы, добавляя знания из документов.\n"
        "Векторный поиск на эмбеддингах позволяет находить похожие фрагменты.\n"
        "Чанкинг нужен, чтобы не терять смысл и не переполнять контекст.",
        encoding="utf-8",
    )

print("Файлы в data:", [p.name for p in data_dir.glob("*.txt")])

## Настройка ключа и base URL

Для AITunnel укажите `OPENAI_API_KEY` и `OPENAI_BASE_URL`. В Colab лучше хранить ключ в переменной окружения.

> Подсказка: можно задать ключ один раз в разделе **Secrets** (значок 🔑 слева в Colab), тогда он будет доступен во всех ячейках.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## Сборка RAG‑пайплайна

Мы:
1) загрузим документы
2) нарежем на чанки
3) создадим эмбеддинги и векторную базу
4) построим retrieval‑chain, который добавляет контекст в ответ

In [ ]:
from pathlib import Path
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

# 1) Загрузка документов
loader = DirectoryLoader("./data", glob="**/*.txt", loader_cls=TextLoader, show_progress=True)
documents = loader.load()

# 2) Чанкинг
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

# 3) Эмбеддинги + векторная база (с хранением на диске)
store = LocalFileStore("./cache/embeddings")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", base_url=os.environ.get("OPENAI_BASE_URL"))
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embeddings, store, namespace="openai")

persist_dir = Path("./db/chroma_rag")
if persist_dir.exists() and any(persist_dir.iterdir()):
    # Загружаем существующую базу с диска
    vectorstore = Chroma(
        persist_directory=str(persist_dir),
        embedding_function=cached_embedder,
    )
else:
    # Создаём базу и сохраняем на диск
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=cached_embedder,
        persist_directory=str(persist_dir),
    )

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 4) Retrieval‑QA chain
llm = ChatOpenAI(
    model="gpt-5.2-chat",
    temperature=0.2,
    max_tokens=256,
    base_url=os.environ.get("OPENAI_BASE_URL"),
)

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
)

question = "Что такое RAG и зачем он нужен?"
answer = rag_chain.invoke({"query": question})
print("Ответ:\n", answer["result"])

## Практические рекомендации

- Если ответы «плывут», снижайте `temperature` и уменьшайте `k`.
- Для длинных документов увеличивайте `chunk_size`.
- Сохраняйте `persist_directory`, чтобы не пересчитывать базу.
- Для качества включайте логирование запросов и результатов.